In [19]:
!pip install azure-identity azure-ai-ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try: 
    credential = DefaultAzureCredential()
    # Check if given credential can get token sucessfully.
    credential.get_token ("http://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzurecredential not work
    credential = InteractiveBrowserCredential(tenant_id="d218e2b7-20ec-4a46-89fd-242b3b84adf2")

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	WorkloadIdentityCredential: WorkloadIdentityCredential authentication unavailable. The workload options are not fully configured. See the troubleshooting guide for more information: https://aka.ms/azsdk/python/identity/workloadidentitycredential/troubleshoot. Missing required arguments: 'tenant_id', 'client_id', 'token_file_path'.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStudioCodeCredential: VisualStudioCodeCredential requires the 'azure-identity-broker' pa

In [20]:
#Get a handle to the workspace
ml_client = MLClient.from_config (credential=credential) 

Found the config file in: .\config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [21]:
%%writefile src/sorvetes-training.py
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#load dataset sorvetes.csv
print("Loading dataset...")
data = pd.read_csv("sorvetes.csv")

#split dataset into features and labels
X, y = sorvetes[['Temperatura']].values, sorvetes['Vendas'].values

#split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#set regularization hyperparameter
reg = 0.01

#train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver='liblinear').fit(X__train, y_train)

#calculate accuracy
y_hat = model.predict(X_test)
acc = np.avarage(y_hat == y_test)
print('Accuracy is', acc)

#calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:,1])
print('AUC: ' + str(auc))


Overwriting src/sorvetes-training.py


In [24]:
from azure.ai.ml import command

#config job
job = command(
    code="./src",
    command="python sorvetes-training.py",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    compute="aml-cluster",
    experiment_name="sorvetes-training",
    display_name="sorvetes-pythonv2-train",
    #description="Train a logistic regression model to predict ice cream sales based on temperature",
    #instance_count=2,
)

#submit job
returned_job = ml_client.create_or_update(job) 
aml_url = returned_job.studio_url
print('Monitor your job at', aml_url)

HttpResponseError: (UserError) Unknown compute target 'aml-cluster'.
Code: UserError
Message: Unknown compute target 'aml-cluster'.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "11700058bcf2b6db68ddf250482f1e24",
        "request": "ca2c76c9f5786ccf"
    }
}Type: Environment
Info: {
    "value": "eastus"
}Type: Location
Info: {
    "value": "eastus"
}Type: Time
Info: {
    "value": "2025-10-02T09:47:40.8435494+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgument",
        "innerError": {
            "code": "UnknownTargetType",
            "innerError": null
        }
    }
}